<a href="https://colab.research.google.com/github/janaat22/NLP_NER/blob/master/NER_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')


In [0]:
pip install sklearn-crfsuite

In [0]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from sklearn_crfsuite import CRF, scorers, metrics
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

In [0]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [0]:


def file_to_list(file_name):  
  training_key_value = []
  word_dict = {}  
  word_list = []
  file1 = open(file_name, 'r')
  Lines = file1.readlines() 
  
  sentence = 1
  for line in Lines:    
    training_key_value = line.split()
    if(len(training_key_value) == 2):
      word = training_key_value[0]
      tag = training_key_value[1]
      word_dict[word] = tag
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      word_list.append(((sentence, word, tagged[0][1], tag))) 
      tags.update(tagged)
    else:
      sentence = sentence + 1
  return word_dict,word_list

 
#Train data
word_dict_train, word_list_train = file_to_list('train.txt')
array_of_words_train = np.array(list(word_dict_train.keys()))
array_of_values_train = np.array(list(word_dict_train.values()))
X_Y_train = np.column_stack((array_of_words_train, array_of_values_train))

#Test data
word_dict_dev,word_list_dev = file_to_list('dev.txt')
array_of_words_dev = np.array(list(word_dict_dev.keys()))
array_of_values_dev = np.array(list(word_dict_dev.values()))
X_Y_dev = np.column_stack((array_of_words_dev, array_of_values_dev))


In [44]:
train_data = pd.DataFrame(word_list_train)
print(train_data.head(n = 4))

dev_data = pd.DataFrame(word_list_dev)
print(dev_data.head(n = 4))

   0                1    2  3
0  1  @SammieLynnsMom   NN  O
1  1         @tg10781   NN  O
2  1             they  PRP  O
3  1             will   MD  O
   0       1   2  3
0  1    STOP  NN  O
1  1    WHAT  WP  O
2  1  YOU'RE  NN  O
3  1   DOING  NN  O


In [61]:
train_data.columns = data.iloc[0]
train_data = train_data[1:]
train_data.columns = ['Sentence','Word','POS','Tag']
train_data = train_data.reset_index(drop=True)
#train_data.head(n=5)

dev_data.columns = data.iloc[0]
dev_data = dev_data[1:]
dev_data.columns = ['Sentence','Word','POS','Tag']
dev_data = dev_data.reset_index(drop=True)
#dev_data.head(n=5)

,Sentence,Word,POS,Tag
0,1,WHAT,WP,O
1,1,YOU'RE,NN,O
2,1,DOING,NN,O
3,1,AND,CC,O
4,1,GO,VB,O


In [62]:
train_data.shape

(46467, 4)

In [0]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [64]:
train_getter = getsentence(train_data)
train_sentences = getter.sentences

dev_getter = getsentence(dev_data)
dev_sentences = getter.sentences

#This is how a sentence will look like. 
print(train_sentences[0])

[('@tg10781', 'NN', 'O'), ('they', 'PRP', 'O'), ('will', 'MD', 'O'), ('be', 'VB', 'O'), ('all', 'DT', 'O'), ('done', 'VBN', 'O'), ('by', 'IN', 'O'), ('Sunday', 'NNP', 'O'), ('trust', 'NN', 'O'), ('me', 'PRP', 'O'), ('*wink*', 'NN', 'O')]


In [0]:
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [76]:
sent2features(train_sentences[3])[0]

{'+1:postag': 'RB',
 '+1:postag[:2]': 'RB',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'perhaps',
 'BOS': True,
 'bias': 1.0,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': '@ls_n',
 'word[-2:]': '_n',
 'word[-3:]': 's_n'}

In [0]:
#Creating the train and test set

X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

X_dev = [sent2features(s) for s in dev_sentences]
y_dev = [sent2labels(s) for s in dev_sentences]

In [0]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [0]:
Model_Roadster = crf.fit(X_train,y_train)

In [73]:
labels = list(crf.classes_)
labels.remove('O')
labels


['B-location',
 'I-location',
 'B-title',
 'I-title',
 'B-company',
 'B-product',
 'B-person',
 'B-other',
 'I-other',
 'B-group',
 'I-group',
 'I-product',
 'I-company',
 'I-person']

In [74]:
y_pred = crf.predict(X_dev)
metrics.flat_f1_score(y_dev, y_pred,
                      average='weighted', labels=labels)

0.9842624520657051

In [75]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

   B-company      1.000     0.959     0.979       171
   I-company      1.000     0.944     0.971        36
     B-group      1.000     0.953     0.976       106
     I-group      1.000     0.988     0.994        84
  B-location      0.995     0.979     0.987       380
  I-location      0.994     0.994     0.994       154
     B-other      0.991     0.929     0.959       225
     I-other      1.000     0.969     0.984       320
    B-person      1.000     0.976     0.988       449
    I-person      1.000     0.995     0.998       215
   B-product      1.000     0.948     0.974        97
   I-product      1.000     1.000     1.000        80
     B-title      1.000     0.956     0.977        68
     I-title      1.000     0.987     0.993        77

   micro avg      0.998     0.971     0.984      2462
   macro avg      0.998     0.970     0.984      2462
weighted avg      0.998     0.971     0.984      2462



# Hyper Parameter Optimization

In [88]:
crf = CRF(
    algorithm='lbfgs',
    max_iterations=10,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=10,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff4c11f3198>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False,
                   scoring=make_scorer(flat_f1_score, average=weighted, l

# Best Parameters

In [89]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.2855989975415824, 'c2': 0.08409780434594832}
best CV score: 0.05702233048689415
model size: 1.00M


In [95]:
crf = rs.best_estimator_
y_pred = crf.predict(X_dev)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   B-company      0.000     0.000     0.000       171
   I-company      0.000     0.000     0.000        36
     B-group      0.000     0.000     0.000       106
     I-group      0.000     0.000     0.000        84
  B-location      0.455     0.013     0.026       380
  I-location      0.000     0.000     0.000       154
     B-other      0.000     0.000     0.000       225
     I-other      0.000     0.000     0.000       320
    B-person      0.400     0.013     0.026       449
    I-person      0.500     0.028     0.053       215
   B-product      0.000     0.000     0.000        97
   I-product      0.000     0.000     0.000        80
     B-title      0.000     0.000     0.000        68
     I-title      0.000     0.000     0.000        77

   micro avg      0.447     0.007     0.014      2462
   macro avg      0.097     0.004     0.007      2462
weighted avg      0.187     0.007     0.013      2462



In [96]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
O      -> O       1.696730
B-person -> I-person 0.744380
I-other -> I-other 0.621639
B-other -> I-other 0.590939
O      -> B-person 0.504542
O      -> B-location 0.443643
B-location -> I-location 0.346572
B-group -> I-group 0.182684
B-title -> I-title 0.164092
B-product -> I-product 0.158782
B-location -> O       0.148248
B-person -> O       0.106718
O      -> B-other 0.092880
O      -> B-company 0.059575
I-location -> I-location 0.055824
I-product -> I-product 0.047667
B-company -> I-company 0.035962
I-title -> I-title 0.033574
I-group -> I-group 0.033435
I-company -> I-company 0.010740

Top unlikely transitions:
B-location -> B-person -0.011673
B-person -> B-location -0.011768
B-person -> I-other -0.012723
B-person -> B-person -0.014765
O      -> B-title -0.041395
I-group -> O       -0.051114
B-other -> O       -0.065789
I-title -> O       -0.085610
I-product -> O       -0.094749
B-group -> O       -0.099142
B-product -> O       -0.100163
I-company -> O       

In [97]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
2.506226 O        bias
0.683610 O        BOS
0.661999 B-person word.istitle()
0.621411 I-other  -1:word.istitle()
0.538803 I-person -1:word.istitle()
0.488431 O        postag[:2]:VB
0.458450 B-person postag[:2]:NN
0.445222 O        -1:postag[:2]:PR
0.442577 O        EOS
0.439300 B-location -1:postag:IN
0.439300 B-location -1:postag[:2]:IN
0.426096 O        postag[:2]:PR
0.406211 O        +1:postag[:2]:PR
0.398361 B-person +1:word.istitle()
0.378617 B-person postag:NN
0.346770 O        postag:PRP
0.346597 O        -1:postag:PRP
0.344704 B-person postag:NNP
0.333849 O        postag[:2]:RB
0.333466 B-location postag[:2]:NN
0.317942 O        postag:RB
0.306274 O        +1:postag:PRP
0.299992 B-location -1:word.lower():in
0.293281 O        +1:postag:NN
0.291796 O        postag:IN
0.291796 O        postag[:2]:IN
0.291168 O        -1:postag[:2]:VB
0.289378 B-location postag:NNP
0.278414 O        -1:postag[:2]:RB
0.277654 O        -1:postag:.

Top negative:
-0.074483 I-title  pos